In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def hamd_model_min_params(delay, baseline, reduction_rate, residual_score, rebound_start=None, rebound_rate=0, rebound_steepness=5, stable_bounce=False, stable_value=None, weeks=8):
    """Simulates HAMD-17 score changes."""
    hamd_scores = []
    for week in range(weeks + 1):
        if week <= delay:  # Keep score at baseline during the delay period
            hamd = baseline
        else:
            effective_week = week - delay #calculate the effective week after delay
            hamd = residual_score + (baseline - residual_score) * np.exp(-reduction_rate * effective_week)

            if rebound_start is not None:
                rebound_factor = 1 / (1 + np.exp(-rebound_steepness * (effective_week - rebound_start)))
                rebound_increase = rebound_rate * (effective_week - rebound_start) * rebound_factor
                hamd += rebound_increase

            if stable_bounce and rebound_start is not None and effective_week >= rebound_start:
                hamd = min(hamd, stable_value)

        hamd_scores.append(int(round(hamd)))
    return hamd_scores

# Scenario Parameters (now with parameter ranges)
scenarios = {
    "Remission: Rapid responders": {"delay": (0, 2), "baseline": (20, 36), "reduction_rate": (0.4, 0.7), "residual_score": (1, 2), "rebound_start": (None,None),"rebound_rate":(0,0),"rebound_steepness":(0,0),"stable_bounce":(False,False),"stable_value":(None,None)},
    "Remission: Slow responders": {"delay": (0, 2), "baseline": (20, 36), "reduction_rate": (0.15, 0.26), "residual_score": (1, 2),"rebound_start": (None,None),"rebound_rate":(0,0),"rebound_steepness":(0,0),"stable_bounce":(False,False),"stable_value":(None,None)},
    "Remission: Temporal remission": {"delay": (0, 2), "baseline": (20, 36), "reduction_rate": (0.25, 0.4), "residual_score": (1, 3), "rebound_start": (3, 6), "rebound_rate": (5, 10), "rebound_steepness":(3,8),"stable_bounce": (True,True), "stable_value": (15,24)},
    "Non-Remission (Slow Responder)": {"delay": (0, 2), "baseline": (20, 36), "reduction_rate": (0.1, 0.2), "residual_score": (10, 18),"rebound_start": (None,None),"rebound_rate":(0,0),"rebound_steepness":(0,0),"stable_bounce":(False,False),"stable_value":(None,None)},
    "Non-responder - flat": {"delay": (0, 2), "baseline": (20, 36), "reduction_rate": (0.01, 0.05), "residual_score": (18, 25),"rebound_start": (None,None),"rebound_rate":(0,0),"rebound_steepness":(0,0),"stable_bounce":(False,False),"stable_value":(None,None)},
    "Non-responder Bounce-Back Stable": {"delay": (0, 2), "baseline": (20, 36), "reduction_rate": (0.2, 0.4), "residual_score": (5, 10), "rebound_start": (4, 7), "rebound_rate": (4, 8), "rebound_steepness":(3,8),"stable_bounce": (True,True), "stable_value": (15,20)},
}

# simulate phenotye of 1000 individuals, with phenotypes of the 8 parameters
n_individuals = 1000
data = []

for scenario_name, params in scenarios.items():
    n_scenario_individuals = n_individuals // len(scenarios)  # Distribute individuals evenly
    if scenario_name == list(scenarios.keys())[-1]:
        n_scenario_individuals = n_individuals - (n_individuals // len(scenarios))*(len(scenarios)-1) #adjust for the last scenario to make the total number of individuals exactly 100

    for i in range(n_scenario_individuals):
        while True:
            individual_params = {}
            for param_name, param_range in params.items():
                if param_range[0] is None:
                    individual_params[param_name] = None
                elif isinstance(param_range[0], bool):
                    individual_params[param_name] = param_range[0]
                else:
                    individual_params[param_name] = np.random.randint(param_range[0], param_range[1] + 1) if isinstance(param_range[0], int) else np.random.uniform(param_range[0], param_range[1])
            param_values = tuple(individual_params.values())
            if param_values not in [tuple(row[2:10]) for row in data]:
                break
        hamd_scores = hamd_model_min_params(**individual_params)
        data.append(["IND_" + str(len(data)+1), scenario_name] + list(individual_params.values()) + hamd_scores)

df = pd.DataFrame(data, columns=['ID', 'Scenario', 'delay', 'baseline', 'reduction_rate', 'residual_score',
                                'rebound_start', 'rebound_rate', 'rebound_steepness', 'stable_bounce', 'stable_value',
                                'week0', 'week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8'])


# Create output directory if it doesn't exist
output_dir = "output_data_with_delay"
os.makedirs(output_dir, exist_ok=True)

# Save the DataFrame to a CSV file
df.to_csv(os.path.join(output_dir, "simulated_hamd_data.csv"), index=False)

# Plotting Individual Trajectories
plt.figure(figsize=(12, 6))
for index, row in df.iterrows():
    plt.plot(range(9), row[['week0', 'week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8']], alpha=0.5)

plt.axhline(y=7, color='blue', linestyle='--', label='Remission Threshold (7)')
plt.axhline(y=20, color='red', linestyle='--', label='Baseline Threshold (20)')

plt.title("Individual HAMD-17 Trajectories")
plt.xlabel("Week")
plt.ylabel("HAMD-17 Score")
plt.legend()
plt.savefig(os.path.join(output_dir, "individual_trajectories.png"))
plt.close()

# Plotting Average Trajectories by Scenario
plt.figure(figsize=(12, 6))
for scenario in df['Scenario'].unique():
    subset = df[df['Scenario'] == scenario]
    average_scores = subset[['week0', 'week1', 'week2', 'week3', 'week4', 'week5', 'week6', 'week7', 'week8']].mean()
    plt.plot(range(9), average_scores, label=scenario, linewidth=2)

plt.axhline(y=7, color='blue', linestyle='--', label='Remission Threshold (7)')
plt.axhline(y=20, color='red', linestyle='--', label='Baseline Threshold (20)')

plt.title("Average HAMD-17 Trajectories by Scenario")
plt.xlabel("Week")
plt.ylabel("Average HAMD-17 Score")
plt.legend()
plt.savefig(os.path.join(output_dir, "trajectories_by_scenario_average.png"))
plt.close()

# Summary Table
summary_table = df['Scenario'].value_counts().reset_index()
summary_table.columns = ['Scenario', 'Count']
summary_table['Percentage'] = (summary_table['Count'] / len(df)) * 100

#Save the summary table to a text file
with open(os.path.join(output_dir, "response_summary.txt"), "w") as f:
    f.write(summary_table.to_string())

print("Data and plots saved to the 'output_data' directory.")
print(summary_table)

Data and plots saved to the 'output_data' directory.
                           Scenario  Count  Percentage
0  Non-responder Bounce-Back Stable    170        17.0
1       Remission: Rapid responders    166        16.6
2        Remission: Slow responders    166        16.6
3     Remission: Temporal remission    166        16.6
4    Non-Remission (Slow Responder)    166        16.6
5              Non-responder - flat    166        16.6
